In [7]:
import bempp.api
from bempp.api.assembly.blocked_operator import BlockedOperator
import math
import numpy as np
import scipy 
import cmath
from numba import objmode
from numpy.linalg import slogdet
from bempp.api.operators.far_field import helmholtz as helmholtz_farfield
import matplotlib.pyplot as plt
import bempp.api.shapes.fractals

In [4]:
h = 0.1
dist = 0.5

sphere1 = bempp.api.shapes.sphere(r = 1, h=h, origin=((dist/2) + 1, 0, 0))
sphere2 = bempp.api.shapes.sphere(r = 0.5, h=h, origin=(-((dist/2) + 0.5), 0, 0))

space_sphere1 = bempp.api.function_space(sphere1, "P", 1)
space_sphere2 = bempp.api.function_space(sphere2, "P", 1)

for N_wn in [20, 30, 40, 50, 60, 70]:

    for ubound_wn in [10]:
        lbound_wn = 0
        quad_points = np.linspace(np.exp(-ubound_wn), np.exp(-lbound_wn), N_wn)
        results = []
        for index2, point2 in enumerate(quad_points):

            wavenumber = -1j * np.log(point2)

            slp11 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere1, space_sphere1, wavenumber)
            slp12 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere1, space_sphere1, wavenumber)
            slp21 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere2 , space_sphere2 , wavenumber)
            slp22 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere2 , space_sphere2 , wavenumber)

            mat11 = slp11.weak_form().A
            mat12 = slp12.weak_form().A
            mat21 = slp21.weak_form().A
            mat22 = slp22.weak_form().A
            mat12_zero = np.zeros((mat11.shape[0],mat12.shape[1]))
            mat21_zero = np.zeros((mat22.shape[0],mat11.shape[1]))

            mat = np.block([[mat11,mat12],[mat21,mat22]])  
            mat11_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat11), np.eye(mat11.shape[0]))
            mat22_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat22), np.eye(mat22.shape[0]))

            inv_free_mat2 = np.block([[mat11_inv,  mat12_zero],
                                  [mat21_zero, mat22_inv]])  

            combined_inverse_free = inv_free_mat2.dot(mat)
            sign, logdet = slogdet(combined_inverse_free)

            results.append(logdet)
        print(N_wn, np.trapz(-np.array(results) / quad_points, quad_points) / (2 * np.pi))

20 0.029483954790765435
30 0.029505105191228474
40 0.029514631613564195
50 0.029519933234768973
60 0.029523263666189396
70 0.029525527743416404


In [5]:
h = 0.1

r1 = 1
r2 = 1

dist = 0.5
sphere1 = bempp.api.shapes.sphere(r = r1, h=h, origin=((dist/2) + r1, 0, 0))
sphere2 = bempp.api.shapes.sphere(r = r2, h=h, origin=(-((dist/2) + r2), 0, 0))

space_sphere1 = bempp.api.function_space(sphere1, "P", 1)
space_sphere2 = bempp.api.function_space(sphere2, "P", 1)

N_wn = 20
ubound_wn = 10
lbound_wn = 0
quad_points = np.linspace(np.exp(-ubound_wn), np.exp(-lbound_wn), N_wn)

m = 20

p = int(m/2)

In [9]:
import krypy
results = []
for index2, point2 in enumerate(quad_points):

    wavenumber = -1j * np.log(point2)

    slp11 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere1, space_sphere1, wavenumber)
    slp12 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere1, space_sphere1, wavenumber)
    slp21 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere2 , space_sphere2 , wavenumber)
    slp22 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere2 , space_sphere2 , wavenumber)

    mat11 = slp11.weak_form().A
    mat12 = slp12.weak_form().A
    mat21 = slp21.weak_form().A
    mat22 = slp22.weak_form().A
    mat12_zero = np.zeros((mat11.shape[0],mat12.shape[1]))
    mat21_zero = np.zeros((mat22.shape[0],mat11.shape[1]))

    mat = np.block([[mat11,mat12],[mat21,mat22]])  
    mat2 = np.block([[mat11,mat12_zero],[mat21_zero,mat22]])
    
    #compute p smallest eigenalues 
    x_s = np.ones(mat.shape[0]) / np.linalg.norm(np.ones(mat.shape[0]))
    rho_s = (x_s.T @ mat @ x_s)/(x_s.T @ mat2 @ x_s)

    result_smallest = []
    for index_ in range(p):

        for k in range(50):
            V_m, H_m = krypy.utils.arnoldi((mat - rho_s*mat2), x_s.reshape(mat.shape[0],1), maxiter = m, ortho ='dmgs')
            A_m = V_m.T @ (mat - rho_s*mat2) @ V_m
            B_m = V_m.T @ mat2 @ V_m
            evalue, evect = scipy.linalg.eigh(A_m, B_m, eigvals_only=False)
            mu_ = sorted(evalue)[index_]
            v_index = list(evalue).index(sorted(evalue)[index_])
            v_ = evect[:,v_index]
            rho_s += mu_
            x_s = V_m @ v_
        result_smallest.append(rho_s)
        
        
    #compute p largest eigenalues
    x_l = np.ones(mat.shape[0]) / np.linalg.norm(np.ones(mat.shape[0]))
    rho_l = (x_l.T @ (-mat) @ x_l)/(x_l.T @ mat2 @ x_l)

    result_largest = []
    for index_ in range(p):

        for k in range(50):
            V_m, H_m = krypy.utils.arnoldi(((-mat) - rho_l*mat2), x_l.reshape(mat.shape[0],1), maxiter = m, ortho ='dmgs')
            A_m = V_m.T @ ((-mat) - rho_l*mat2) @ V_m
            B_m = V_m.T @ mat2 @ V_m
            evalue, evect = scipy.linalg.eigh(A_m, B_m, eigvals_only=False)
            mu_ = sorted(evalue)[index_]
            v_index = list(evalue).index(sorted(evalue)[index_])
            v_ = evect[:,v_index]
            rho_l += mu_
            x_l = V_m @ v_
        result_largest.append(-rho_l)    
        
    method2_logdet = 0
    for j in range(p):
        method2_logdet += np.log(result_smallest[j])
        method2_logdet += np.log(result_largest[j])
    print(method2_logdet)
    results.append(method2_logdet)
print(np.trapz(-np.array(results)/quad_points, quad_points)/(2*np.pi))

-0.0001786022482530126
0.016429405046120556
-0.06901726930146035
-0.05391620976677878
-0.09915993009628253
0.0015054490594789412
-0.04788917785521307
-0.15420735397487648
-0.06180904665773408
-0.18795518088472404
-0.0450682616207302
-0.14875124963129335
-0.1772078917124066
-0.19736557857851542
-0.21231960553580448
-0.24632322872815082
-0.2724493263851601
-0.2813692636211818
-0.20559858473874015
-0.5014468724657144
0.05736258089446505
